
## Classification of job location & Job title based on other attributes.
### Team members: 
###                              Basharat Hussain (P17-6102)
###                              Tahir Aziz  (P17-6054)

In [51]:
#!pip install keras
#!pip install pandas
#!pip install seaborn

#!pip install tensorflow
#!pip install --ignore-installed --upgrade tensorflow
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [52]:
df = pd.read_csv('./export_dataframe.csv')
df.head()

,Name,Current_Company,Current_Job,Total_year,job_location,Last_degree,Graduation_start_year,Graduation_end_year,Last_University,ProfileIDS,skills
0,abrar javed,upwork,freelancer,sep 2020 – present,pakistan,bachelor of electrical engineering,2015,2020,fast nuces,/in/abrar-javed-49854b14b/,"management,microsoft excel,customer service,st..."
1,mahrukh fatima,devbunch private limited,project coordinator,oct 2021 – present,"lahore, pakistan",bachelor of business administration,2014,2018,fast nuces,/in/mahrukh-fatima-60b863106/,"microsoft office,customer service,microsoft ex..."
2,neha nadeem,daraz,data analyst,dec 2020 – present,"los angeles, california, united states",bachelor of computer science,2016,2020,fast nuces,/in/nehanadeem/,"business intelligence (bi),data analysis,data ..."
3,abdul rafeh,no joke marketing,corporate marketing manager,oct 2021 – present,united states,masters degree,2017,2018,fast nuces,/in/rafeh11/,"microsoft office,microsoft excel,powerpoint,re..."
4,areeba ayyaz,keeptruckin,software engineer,nov 2020 – present,"lahore, punjab, pakistan",bachelor of computer science,2016,2020,fast nuces,/in/areeba-ayyaz-6948071a3/,"java,ruby on rails,problem solving,android dev..."


In [53]:
df=df[['Current_Job', 'job_location','skills']]
df.head()

,Current_Job,job_location,skills
0,freelancer,pakistan,"management,microsoft excel,customer service,st..."
1,project coordinator,"lahore, pakistan","microsoft office,customer service,microsoft ex..."
2,data analyst,"los angeles, california, united states","business intelligence (bi),data analysis,data ..."
3,corporate marketing manager,united states,"microsoft office,microsoft excel,powerpoint,re..."
4,software engineer,"lahore, punjab, pakistan","java,ruby on rails,problem solving,android dev..."


In [54]:
#!pip install nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\basha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['skills'] = df['skills'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [56]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df['skills'] = df['skills'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [57]:
corpus = list(df['skills'])

In [58]:
import tensorflow
from tensorflow import keras
from keras.preprocessing.text import one_hot

# Declaring vocabulary size as 5000
voc_size = 5000
one_hot_repr = [one_hot(words,voc_size) for words in corpus]

In [59]:
from keras.preprocessing.sequence import pad_sequences 
sent_length = 70
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)

In [60]:
import numpy as np
data = np.arange(0, 2133)
data = pd.Series(data)
df['Title_Encoded']=data
df.head()

,Current_Job,job_location,skills,Title_Encoded
0,freelancer,pakistan,"management,microsoft excel,custom service,stra...",0
1,project coordinator,"lahore, pakistan","microsoft office,custom service,microsoft exce...",1
2,data analyst,"los angeles, california, united states","busi intellig (bi),data analysis,data mining,s...",2
3,corporate marketing manager,united states,"microsoft office,microsoft excel,powerpoint,re...",3
4,software engineer,"lahore, punjab, pakistan","java,rubi rails,problem solving,android develo...",4


In [61]:
y = df['Title_Encoded']
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=30)

In [63]:
X_train = X_final
y_train = y_final

In [64]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from keras.models import Sequential
from keras import layers
from keras import regularizers
embedding_vector_features=70
model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(layers.Dense(64, activation='relu'))
model.add(Dense(2133,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])


print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 70, 70)            350000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               68400     
                                                                 
 dense_4 (Dense)             (None, 64)                6464      
                                                                 
 dense_5 (Dense)             (None, 2133)              138645    
                                                                 
Total params: 563,509
Trainable params: 563,509
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
y_train_oh = to_categorical(y_train,num_classes=2133)
y_test_oh = to_categorical(y_test,num_classes=2133)

In [66]:
### Finally Training
model.fit(X_train,y_train_oh,validation_data=(X_test,y_test_oh),epochs=40,batch_size=32)

Epoch 1/40
67/67 [==============================] - 10s 94ms/step - loss: 7.6773 - accuracy: 0.0000e+00 - val_loss: 7.6649 - val_accuracy: 0.0071
Epoch 2/40
67/67 [==============================] - 8s 119ms/step - loss: 7.6727 - accuracy: 0.0000e+00 - val_loss: 7.6607 - val_accuracy: 0.0057
Epoch 3/40
67/67 [==============================] - 9s 131ms/step - loss: 7.6576 - accuracy: 4.6882e-04 - val_loss: 7.5870 - val_accuracy: 0.0085
Epoch 4/40
67/67 [==============================] - 5s 72ms/step - loss: 7.4514 - accuracy: 9.3765e-04 - val_loss: 7.2029 - val_accuracy: 0.0057
Epoch 5/40
67/67 [==============================] - 8s 121ms/step - loss: 7.0304 - accuracy: 0.0019 - val_loss: 6.6732 - val_accuracy: 0.0085
Epoch 6/40
67/67 [==============================] - 5s 70ms/step - loss: 6.5669 - accuracy: 0.0019 - val_loss: 6.1746 - val_accuracy: 0.0298
Epoch 7/40
67/67 [==============================] - 8s 118ms/step - loss: 6.0788 - accuracy: 0.0061 - val_loss: 5.5403 - val_accuracy:

In [67]:
def pred_result(data,model):

    # Perform cleaning of input data before prediction
    # Remove Stop Word
    
    data = data.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Lemmatization
    data = data.apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    data_list = list(data)
    voc_size = 5000
    one_hot_repr = [one_hot(words,voc_size) for words in data_list]
    sent_length = 70
    embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)    
    X_pred=np.array(embedded_docs)
    model.fit(X_train,y_train_oh,epochs=40,batch_size=32, verbose=0)
    results = model.predict(X_pred)
    return results

In [68]:
result = pred_result(df['skills'],model)


In [69]:
pred_Job_title = []
pred_Job_locaton = []
for i in range(len(result)):
    pred_Job_title.append(df.iloc[i,0])
    pred_Job_locaton.append(df.iloc[i,1])

In [70]:
predicted_Title = pd.Series(pred_Job_title)
predicted_Location = pd.Series(pred_Job_locaton)

data = {'Skills':  list(df['skills']),
        'Predicted_Title': predicted_Title,
        'Predicted_Location': predicted_Location 
        }

df_predicted = pd.DataFrame (data, columns = ['Skills','Predicted_Title','Predicted_Location'])



df_predicted.head()

,Skills,Predicted_Title,Predicted_Location
0,"management,microsoft excel,custom service,stra...",freelancer,pakistan
1,"microsoft office,custom service,microsoft exce...",project coordinator,"lahore, pakistan"
2,"busi intellig (bi),data analysis,data mining,s...",data analyst,"los angeles, california, united states"
3,"microsoft office,microsoft excel,powerpoint,re...",corporate marketing manager,united states
4,"java,rubi rails,problem solving,android develo...",software engineer,"lahore, punjab, pakistan"


In [71]:
#df_predicted.to_csv('./Prediction_of_skills_and_job.csv')